In [29]:
# -*- coding:utf-8 -*-
import pymongo
'''
    房天下抓取的，天河区商铺出租，出售信息。在本文档进行清洗和处理。
'''
START_LNG  = 113.297942
END_LNG = 113.466578
START_LAT = 23.093353
END_LAT = 23.263226
LNG_GRIDS = 20
LAT_GRIDS = 18
LNG_DELTA = round((END_LNG - START_LNG) / LNG_GRIDS, 7)
LAT_DELTA = round((END_LAT - START_LAT) / LAT_GRIDS, 7)

# 不在范围内则返回 -1
# 边界值时会出现随机现象，和python的浮点处理不精确有关
# 只要保证插入和读取都用此函数，不影响分析正确性
# 查表获取ID，太繁琐低效，后续如有需要则改为查表
def get_grid_id(lng, lat):
    column = round((lng - START_LNG),7) // LNG_DELTA + 1
    row = round((END_LAT - lat),7) // LAT_DELTA + 1
    if column < 1 or column > 20: return -1
    if row < 1 or row > 18: return -1
    ret = 'R%dC%d' % (row, column)
    return ret 

def get_int_from_numbegin(numstr):
    rslt = []
    for char in numstr.strip():
        if char.isdigit():
            rslt.append(char)
        else:
            break
    tmp = ''.join(rslt) 
    return int(tmp)

def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn

def get_crawl_shop_rent():
    db = get_monggo_db()
    return db['crawl_fang_shop_rent']

def get_crawl_shop_sale():
    db = get_monggo_db()
    return db['crawl_fang_shop_sale']

def get_etl_shop_rent():
    db = get_monggo_db()
    return db['etl_fang_shop_rent']

def get_etl_shop_sale():
    db = get_monggo_db()
    return db['etl_fang_shop_sale']

def get_result_coll():
    db = get_monggo_db()
    return db['result_fang_shop_rentsale']

def etl_shop_rent():
    dst_coll = get_etl_shop_rent()
    src_coll = get_crawl_shop_rent()
    cursor = src_coll.find()
    for cur in cursor:
        title = cur.get('title').replace('�O', '平米')
        title = cur.get('title').replace('�搬�村', '长湴岑村')
        region = cur.get('region').replace('�Z','璟')
        area = get_int_from_numbegin(cur.get('area'))
        price = get_int_from_numbegin(cur.get('price'))
        lng = round(float(cur.get('lng')), 7)
        lat = round(float(cur.get('lat')), 7)        
        cur.pop('_id')
        cur['title'] = title
        cur['region'] = region
        cur['area'] = area
        cur['price'] = price
        cur['lng'] = lng
        cur['lat'] = lat
        if price < 50 or price > 2000:
            continue        
        dst_coll.update_one({'url': cur.get('url')}, {'$set': cur}, True)
        print('etl_shop_rent ended')
        
def etl_shop_sale():
    dst_coll = get_etl_shop_sale()
    src_coll = get_crawl_shop_sale()
    cursor = src_coll.find()
    for cur in cursor:
        try:
            title = cur.get('title').replace('�O', '平米')
            title = cur.get('title').replace('�搬�村', '长湴岑村')
            region = cur.get('region').replace('�Z','璟')
            area = get_int_from_numbegin(cur.get('area'))
            price = get_int_from_numbegin(cur.get('price'))
            lng = round(float(cur.get('lng')), 7)
            lat = round(float(cur.get('lat')), 7)        
            cur.pop('_id')
            cur['title'] = title
            cur['region'] = region
            cur['area'] = area
            cur['price'] = price
            cur['lng'] = lng
            cur['lat'] = lat        
            dst_coll.update_one({'url': cur.get('url')}, {'$set': cur}, True)
        except Exception as e:
            print(e)
    print('etl_shop_sale ended')
    
def export2result():
    rent_coll = get_etl_shop_rent()
    sale_coll = get_etl_shop_sale()
    dst_coll = get_result_coll()
    # 导出 rent 表
    print('exporting rent...')
    cursor = rent_coll.find()
    for cur in cursor:
        lng = cur.get('lng')
        lat = cur.get('lat')
        gridid = get_grid_id(lng, lat)
        if gridid == -1:
            print('gridid = -1:',cur.get('region'))
            continue
        toset_doc = {
            'rent_or_sale': 'rent',
            'title': cur.get('title'),
            'addr': cur.get('addr'),
            'name': cur.get('region'),
            'url': cur.get('url'),
            'area': cur.get('area'),
            'price': cur.get('price'),
            'lng': lng,
            'lat': lat,
            'gridid': gridid,
        }
        print(gridid)
        dst_coll.update_one({'url': cur.get('url')}, {'$set': toset_doc}, True)
    # 导出 sale 表
    print('exporting sale...')
    cursor = sale_coll.find()
    for cur in cursor:
        lng = cur.get('lng')
        lat = cur.get('lat')
        gridid = get_grid_id(lng, lat)
        if gridid == -1:
            print('gridid = -1:',cur.get('region'))
            continue
        toset_doc = {
            'rent_or_sale': 'sale',
            'title': cur.get('title'),
            'addr': cur.get('addr'),
            'name': cur.get('region'),
            'url': cur.get('url'),
            'area': cur.get('area'),
            'price': cur.get('price'),
            'lng': lng,
            'lat': lat,
            'gridid': gridid,
        }
        print(gridid)
        dst_coll.update_one({'url': cur.get('url')}, {'$set': toset_doc}, True)
    print('export2result ended')

def append_fields2result_grids():
    src_coll = get_result_coll()
    db = get_monggo_db()
    dst_coll = db['result_grids']
    grids = dst_coll.distinct('gridid')
    for grid in grids:
        cursor = src_coll.find({'gridid': grid, 'rent_or_sale': 'rent'})
        count = 0
        for cur in cursor:
            count += 1
        dst_coll.update_one({'gridid': grid}, {'$set': {'shoprent_count': count}})
        
    for grid in grids:
        cursor = src_coll.find({'gridid': grid, 'rent_or_sale': 'sale'})
        count = 0
        for cur in cursor:
            count += 1
        dst_coll.update_one({'gridid': grid}, {'$set': {'shopsale_count': count}})
    print('append_fields2result_grids ended')    

if __name__ == '__main__':
#     etl_shop_rent()
#     etl_shop_sale()
#     export2result()
    append_fields2result_grids()
    
    


append_fields2result_grids ended
